In [1]:
%matplotlib inline

In [ ]:
!pip install lightly

In [3]:
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly
from torchmetrics import F1

In [4]:
num_workers = 8
batch_size = 512
memory_bank_size = 4096
seed = 1

In [5]:
path_to_train = '../input/bwcolor39kbw/MNIST-Data-BW&Color-39k-BW/Train'
path_to_test = '../input/bwcolor39kbw/MNIST-Data-BW&Color-39k-BW/Val'

In [ ]:
pl.seed_everything(seed)

In [7]:
collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=28,
    gaussian_blur=0.,
)

In [8]:
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

dataset_train_moco = lightly.data.LightlyDataset(
    input_dir=path_to_train
)
dataset_train_classifier = lightly.data.LightlyDataset(
    input_dir=path_to_train,
    transform=train_classifier_transforms
)

dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_test,
    transform=test_transforms
)

In [9]:
dataloader_train_moco = torch.utils.data.DataLoader(
    dataset_train_moco,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)

dataloader_train_classifier = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [10]:
class MocoModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        resnet = lightly.models.ResNetGenerator('resnet-18', 1, num_splits=8)
        backbone = nn.Sequential(
            *list(resnet.children())[:-1],
            nn.AdaptiveAvgPool2d(1),
        )

        self.resnet_moco = \
            lightly.models.MoCo(backbone, num_ftrs=512, m=0.99, batch_shuffle=True)

        self.criterion = lightly.loss.NTXentLoss(
            temperature=0.1,
            memory_bank_size=memory_bank_size)

    def forward(self, x):
        self.resnet_moco(x)
    def custom_histogram_weights(self):
        for name, params in self.named_parameters():
            self.logger.experiment.add_histogram(
                name, params, self.current_epoch)

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        y0, y1 = self.resnet_moco(x0, x1)
        loss = self.criterion(y0, y1)
        self.log('train_loss_ssl', loss)
        return loss

    def training_epoch_end(self, outputs):
        self.custom_histogram_weights()


    def configure_optimizers(self):
        optim = torch.optim.SGD(self.resnet_moco.parameters(), lr=6e-2,
                                momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

In [11]:
class Classifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()

        self.resnet_moco = model
        for p in self.resnet_moco.parameters():
            p.requires_grad = False
        self.fc = nn.Linear(512, 10)

        self.accuracy = pl.metrics.Accuracy()
        self.f1 = F1(num_classes = 10)

    def forward(self, x):
        with torch.no_grad():
            y_hat = self.resnet_moco.backbone(x).squeeze()
            y_hat = nn.functional.normalize(y_hat, dim=1)
        y_hat = self.fc(y_hat)
        return y_hat

    # We provide a helper method to log weights in tensorboard
    # which is useful for debugging.
    def custom_histogram_weights(self):
        for name, params in self.named_parameters():
            self.logger.experiment.add_histogram(
                name, params, self.current_epoch)

    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log('train_loss_fc', loss)
        return loss

    def training_epoch_end(self, outputs):
        self.custom_histogram_weights()

    def validation_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.forward(x)
        y_hat = torch.nn.functional.softmax(y_hat, dim=1)
        self.accuracy(y_hat, y)
        
        self.log('val_acc', self.accuracy.compute(),
                 on_epoch=True, prog_bar=True)
        self.log('val_f1',self.f1(y_hat, y),
                on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.fc.parameters(), lr=30.)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

In [ ]:
model = MocoModel()
trainer = pl.Trainer(max_epochs=50, gpus=1,
                     progress_bar_refresh_rate=10)
trainer.fit(
    model,
    dataloader_train_moco
)

In [ ]:
model.eval()

classifier = Classifier(model.resnet_moco)
trainer = pl.Trainer(max_epochs=50, gpus=gpus,
                     progress_bar_refresh_rate=10)
trainer.fit(
    classifier,
    dataloader_train_classifier,
    dataloader_test
)